# 🥋 Lekcja 7: Named Tensors (Koniec z magicznymi liczbami)

Standardowe tensory są "ślepe". Mają wymiary 0, 1, 2...
Inżynier musi pamiętać w głowie: *"Okej, 0 to Batch, 1 to Czas, 2 to Cechy"*.
Jeden błąd w `permute(0, 2, 1)` i model uczy się śmieci.

**Named Tensors** pozwalają nadać etykiety wymiarom.
Zamiast `img.mean(2)`, piszesz `img.mean('width')`.

To sprawia, że kod jest:
1.  **Czytelny:** Nie potrzebujesz komentarzy `# [B, C, H, W]`.
2.  **Bezpieczny:** PyTorch sprawdzi, czy dodajesz pasujące do siebie wymiary (np. nie pozwoli dodać 'height' do 'width').

In [1]:
import torch

# 1. Tworzymy tensor z nazwami
# names=(...) definiuje nazwy dla każdego wymiaru
# (Batch, Channels, Height, Width)
img = torch.randn(4, 3, 32, 32, names=('N', 'C', 'H', 'W'))

print(f"Tensor shape: {img.shape}")
print(f"Nazwy wymiarów: {img.names}")

Tensor shape: torch.Size([4, 3, 32, 32])
Nazwy wymiarów: ('N', 'C', 'H', 'W')


C:\Users\takze\AppData\Local\Temp\ipykernel_23580\6474613.py:6: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/core/TensorImpl.h:1974.)
  img = torch.randn(4, 3, 32, 32, names=('N', 'C', 'H', 'W'))


## Operacje po nazwach (Name-based Operations)

Zapomnij o indeksach. Chcesz usunąć kolor (zrobić grayscale)?
Sumujesz po kanale `'C'`. Nie obchodzi Cię, czy 'C' jest na pozycji 1 (PyTorch) czy 3 (TensorFlow/Keras).

In [2]:
# Tradycyjnie (Ryzykowne - musisz wiedzieć, gdzie jest C)
# gray = img.mean(1)

# Named Tensor (Bezpieczne)
gray_named = img.mean('C')

print(f"Po uśrednieniu 'C': {gray_named.shape}")
print(f"Pozostałe nazwy: {gray_named.names}")

# Inny przykład: Max pooling po przestrzeni (H, W)
max_val = img.flatten(['H', 'W'], 'spatial').max('spatial')
# flatten(['H', 'W'], 'spatial') -> scala H i W w nowy wymiar o nazwie 'spatial'

print(f"Max po przestrzeni: {max_val.values.shape}")
print(f"Nazwy: {max_val.values.names}")

Po uśrednieniu 'C': torch.Size([4, 32, 32])
Pozostałe nazwy: ('N', 'H', 'W')
Max po przestrzeni: torch.Size([4, 3])
Nazwy: ('N', 'C')


## Bezpieczeństwo: Broadcasting po nazwach

To jest "killer feature".
W zwykłych tensorach broadcasting działa "od prawej do lewej".
W Named Tensors broadcasting działa **po nazwach**.

Jeśli spróbujesz dodać tensor z wymiarem `('H', 'W')` do tensora `('W', 'H')`, PyTorch **automatycznie je obróci**, żeby pasowały, albo rzuci błędem, jeśli nazwy się gryzą.

In [3]:
# Tensor A: [N, C]
x = torch.randn(3, 5, names=('N', 'C'))

# Tensor B: [C, N] (Odwrotna kolejność!)
y = torch.randn(5, 3, names=('C', 'N'))

# Klasycznie: x + y
# Błąd lub (co gorsza) błędny wynik przez broadcasting (3,5) + (5,3) -> (5,3,5) w numpy?
# Sprawdźmy w Named Tensors:

try:
    z = x + y
    print("✅ SUKCES! PyTorch dopasował wymiary po nazwach.")
    print(f"Wynik shape: {z.shape}")
    print(f"Wynik names: {z.names}")
except RuntimeError as e:
    print(f"❌ Błąd: {e}")

# Przykład błędu: Próba dodania czegoś niepasującego
try:
    # Tensor z wymiarem 'W' (Width) zamiast 'C'
    z_bad = torch.randn(5, names=('W',))
    out = x + z_bad
except RuntimeError as e:
    print("\n🚫 Ochrona przed błędem logicznym:")
    print(e)
    print("(Nie pozwolił dodać 'Width' do 'Channel', mimo że rozmiar 5 pasował!)")

❌ Błąd: Error when attempting to broadcast dims ['N', 'C'] and dims ['C', 'N']: dim 'C' and dim 'N' are at the same position from the right but do not match.

🚫 Ochrona przed błędem logicznym:
Error when attempting to broadcast dims ['N', 'C'] and dims ['W']: dim 'C' and dim 'W' are at the same position from the right but do not match.
(Nie pozwolił dodać 'Width' do 'Channel', mimo że rozmiar 5 pasował!)


## Zmiana nazw i Powrót do normalności

Czasami musisz zmienić nazwę (np. po spłaszczeniu) albo usunąć nazwy, żeby wrzucić tensor do warstwy `nn.Conv2d` (bo stare warstwy PyTorcha jeszcze nie obsługują nazw w 100%).

*   `rename()`: Zmienia etykiety.
*   `rename(None)`: Usuwa nazwy (robi zwykły tensor).
*   `align_to()`: Wymusza konkretną kolejność (jak `permute`, ale po nazwach).

In [4]:
# Align_to (Permute po ludzku)
# Chcemy format dla Matplotlib: (H, W, C)
img_hwc = img.align_to('N', 'H', 'W', 'C')

print(f"Przed align: {img.shape}")
print(f"Po align:    {img_hwc.shape}")

# Usuwanie nazw (Drop names)
img_unnamed = img.rename(None)
print(f"\nBez nazw: {img_unnamed.names}")
print("Teraz można to wrzucić do starych funkcji np. nn.Conv2d")

Przed align: torch.Size([4, 3, 32, 32])
Po align:    torch.Size([4, 32, 32, 3])

Bez nazw: (None, None, None, None)
Teraz można to wrzucić do starych funkcji np. nn.Conv2d


## 🥋 Black Belt Summary

**Named Tensors** to potężne narzędzie do debugowania i pisania bezpiecznego kodu, ale są wciąż w fazie **Experimental**.

**Kiedy używać?**
1.  **Preprocessing danych:** Gdy żonglujesz wymiarami (audio, video) i łatwo się pogubić.
2.  **Skomplikowane operacje:** Redukcje, normalizacje, gdzie łatwo pomylić oś.
3.  **Debugowanie:** Gdy dostajesz błąd kształtów i nie wiesz dlaczego.

**Kiedy NIE używać?**
1.  Wewnątrz standardowych warstw `nn.Module` (np. `nn.Linear`), bo one często usuwają nazwy.
2.  Gdy zależy Ci na maksymalnej kompatybilności ze starym kodem.

To kończy **Moduł 1: Tensory**. Zrozumiałeś pamięć, broadcasting, einsum i nazwy. Jesteś gotowy na **Moduł 2: Autograd Internals**.